In [2]:
from datasets import Dataset, DatasetDict, IterableDataset, IterableDatasetDict
from datasets import load_dataset

raw_datasets = load_dataset('tomaarsen/MultiCoNER', 'multi')

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 168300
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 8800
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 471911
    })
})